# Market Data Analysis applying PPCA
I thought I would try applying a Principal Component Analysis to the market data - to see if this allowed the prediction problem to be reduced any, and the market data to be shared across assets. 

This starts the same way as every Kernel - load libraries, load the data feathers and look at a few rows of the data.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')
market_train_df = env.get_training_data()[0]

In [ ]:
market_train_df.head()

## Probabalistic Principle Component Analysis

Next we need to define the PCA class. Because we are working with data containing (a whole lot) of missing data, the standard PCA approach won't work and we need to look at a probabilistic PCA. This works iteratively - estimating values for the missing data, using those to perform a PCA and repeating until convergence.

The implementation I have used is a small adaptation of the implementation by Allen Tran at:
[https://github.com/allentran/pca-magic/](https://github.com/allentran/pca-magic/)

Thanks to Allen for making this available on a permissive license.

In [ ]:
import os

import numpy as np
from scipy.linalg import orth


class PPCA():

    def __init__(self):

        self.raw = None
        self.data = None
        self.C = None
        self.means = None
        self.stds = None
        self.eig_vals = None

    def _standardize(self, X):

        if self.means is None or self.stds is None:
            raise RuntimeError("Fit model first")

        return (X - self.means) / self.stds

    def fit(self, data, d=None, tol=1e-4, min_obs=10, verbose=False):

        self.raw = data
        self.raw[np.isinf(self.raw)] = np.max(self.raw[np.isfinite(self.raw)])

        valid_series = np.sum(~np.isnan(self.raw), axis=0) >= min_obs

        data = self.raw[:, valid_series].copy()
        N = data.shape[0]
        D = data.shape[1]

        self.means = np.nanmean(data, axis=0)
        self.stds = np.nanstd(data, axis=0)

        data = self._standardize(data)
        observed = ~np.isnan(data)
        missing = np.sum(~observed)
        data[~observed] = 0

        # initial

        if d is None:
            d = data.shape[1]
        
        if self.C is None:
            C = np.random.randn(D, d)
        else:
            C = self.C
        CC = np.dot(C.T, C)
        X = np.dot(np.dot(data, C), np.linalg.inv(CC))
        recon = np.dot(X, C.T)
        recon[~observed] = 0
        ss = np.sum((recon - data)**2)/(N*D - missing)

        v0 = np.inf
        counter = 0

        while True:

            Sx = np.linalg.inv(np.eye(d) + CC/ss)

            # e-step
            ss0 = ss
            if missing > 0:
                proj = np.dot(X, C.T)
                data[~observed] = proj[~observed]
            X = np.dot(np.dot(data, C), Sx) / ss

            # m-step
            XX = np.dot(X.T, X)
            C = np.dot(np.dot(data.T, X), np.linalg.pinv(XX + N*Sx))
            CC = np.dot(C.T, C)
            recon = np.dot(X, C.T)
            recon[~observed] = 0
            ss = (np.sum((recon-data)**2) + N*np.sum(CC*Sx) + missing*ss0)/(N*D)

            # calc diff for convergence
            det = np.log(np.linalg.det(Sx))
            if np.isinf(det):
                det = abs(np.linalg.slogdet(Sx)[1])
            v1 = N*(D*np.log(ss) + np.trace(Sx) - det) \
                + np.trace(XX) - missing*np.log(ss0)
            diff = abs(v1/v0 - 1)
            if verbose:
                print('\rAt iteration {} the diff is {:8.6f} (target {})'.format(counter,diff,tol),end='')
            if (diff < tol) and (counter > 5):
                break

            counter += 1
            v0 = v1


        C = orth(C)
        vals, vecs = np.linalg.eig(np.cov(np.dot(data, C).T))
        order = np.flipud(np.argsort(vals))
        vecs = vecs[:, order]
        vals = vals[order]

        C = np.dot(C, vecs)

        # attach objects to class
        self.C = C
        self.data = data
        self.eig_vals = vals
        self._calc_var()

    def transform(self, data=None):

        if self.C is None:
            raise RuntimeError('Fit the data model first.')
        if data is None:
            return np.dot(self.data, self.C)
        missing = np.isnan(data)
        #Obtain mean of columns as you need, nanmean is just convenient.
        it = 0
        if np.sum(missing) > 0:
            col_mean = np.nanmean(data, axis=0)
            data[missing] = np.take(col_mean, np.where(missing)[0])
            change = 1
            while(change>1e-3):
                CC = np.dot(self.C.T, self.C)
                X = np.dot(np.dot(data, self.C), np.linalg.inv(CC))
                proj = np.dot(X, self.C.T)
                change = np.max(np.abs(data[missing]-proj[missing]))
                print('\rIteration {}. Change is {:6.4f}'.format(it,change),end='')
                data[missing] = proj[missing]
                it += 1
        return np.dot(data, self.C)

    def _calc_var(self):

        if self.data is None:
            raise RuntimeError('Fit the data model first.')

        data = self.data.T

        # variance calc
        var = np.nanvar(data, axis=1)
        total_var = var.sum()
        self.var_exp = self.eig_vals.cumsum() / total_var

    def save(self, fpath):

        np.save(fpath, self.C)
        
    def load(self, fpath):

        assert os.path.isfile(fpath)

        self.C = np.load(fpath)

Before we do any training, lets clean up the data some - removing outliers and and changing the date-time values to simple dates. We already have gaps in our data - so making some more won't hurt much. We drop anything which shows prices on the day changing by more than a factor of 5, 1 days absolute returns greater than 0.5 and 10 day absolute returns greater than 1.5 (approx .5 * sqrt(10)).

In [ ]:
market_train_df['assetCode'].count()

In [ ]:
market_train_df.drop(market_train_df[(market_train_df['open']>market_train_df['close']*5) | (market_train_df['close']>market_train_df['open']*5)].index,inplace=True)
market_train_df.drop(market_train_df[(np.abs(market_train_df['returnsOpenPrevRaw1'])>0.5) | (np.abs(market_train_df['returnsClosePrevRaw1'])>0.5)].index,inplace=True)
market_train_df.drop(market_train_df[(np.abs(market_train_df['returnsOpenPrevRaw10'])>1.5) | (np.abs(market_train_df['returnsClosePrevRaw10'])>1.5)].index,inplace=True)

In [ ]:
market_train_df['time']=market_train_df['time'].dt.floor('D')

In [ ]:
market_train_df['assetCode'].count()

And now we need to convert the relevant dataframe to a data matrix (including NaNs and all) for use in our PPCA. In theory this should save the results - but I can't make that work.

I create 128 components from the data - seems like 100 doesn't give enough accuracy, but 200 may be beyond the point of diminishing returns.  Something of a question of taste however.

In [ ]:
def DfToData(df,valueCol):
    data = df.pivot(index='time',columns='assetCode',values=valueCol).values
    asset_means = np.nanmean(data,axis=0)
    data -= asset_means
    return data

dataList = []
dataList.append(DfToData(market_train_df,'returnsClosePrevRaw10'))
dataList.append(DfToData(market_train_df,'returnsClosePrevMktres10'))
data = np.concatenate(dataList,axis=0)

data.shape
ppca = PPCA()

ppca.fit(data, d=128, verbose=True,min_obs=1)
ppca.save('ppca.np')

Now one of the things that would be really useful here would be if the PPCA told us something fundamental about the relationship between assets - not just the relationship between 10 day returns. To test if this is the case, we will need to apply the PPCA transform to Series from the DataFrame that are not the one used for training. 

In [ ]:
# A function to transform a column from the  using a PPCA which was trained on a different column
def PpcaTransform(df,column,ppca):
    return ppca.transform(DfToData(df,column))

In [ ]:
print('Residual 1 day opening price returns')
Open1d = PpcaTransform(market_train_df,'returnsOpenPrevRaw1',ppca)
print('\n\nResidual 1 day closing price returns')
Close1d = PpcaTransform(market_train_df,'returnsClosePrevRaw1',ppca)
print('\n\nResidual 10 day opening price returns')
Open10d = PpcaTransform(market_train_df,'returnsOpenPrevRaw10',ppca)
print('\n\nResidual 10 day closing price returns')
Close10d = PpcaTransform(market_train_df,'returnsClosePrevRaw10',ppca)
print('\n\nDaily volumes')
Volume = PpcaTransform(market_train_df,'volume',ppca)
print('\n\nThe target forward-looking returns')
Targets = PpcaTransform(market_train_df,'returnsOpenNextMktres10',ppca)

In [ ]:
Market_Features = np.concatenate([Open1d,Close1d,Open10d,Close10d,Volume],axis=1)
np.save('Features.np',Market_Features)
np.save('Targets.np',Targets)

## Visualisation

OK - all very textual so far, let's look at some images.

First things first - can we see any significant structure in the principle components returned?

In [ ]:
fig,ax=plt.subplots(figsize=(30,10))
im=ax.imshow(ppca.C,aspect='auto',cmap='RdBu')

plt.colorbar(im)

So, clearly a consistent bias on the first component (to the negative - as it happens) otherwise hard to discern much. In a first pass there were some assets that seem to be particularly strong on quite a few components - but those have disappeared with better datafiltering.

Next, what do the PPCA timeseries look like for the returns we used for training?

In [ ]:
plt.figure(figsize=(30, 15))
labels = range(ppca.C.shape[1])
for values,label in zip(Close10d.T,labels):
    plt.plot(values,label=label)
plt.legend()

For the most part that is so much noise. A bit of structure - some points clearly much more volatile than others. The first component (the blue line) showing the largest moves, as would be normal for a PCA.

How about the volume transformed with the same analysis?

In [ ]:
plt.figure(figsize=(30, 15))
plt.plot(Volume)

This would seem to have less obvious output structure. Possibly still some dominance from the first component - but much harder to tell. But then - what would really be expected from a PCA of daily volumes?

We'd better check that the transformation is actually working to fit the market. Let's try that for an arbitrary asset - showing how much of the assets movement can be explained by the PCA and how much is residual noise.

In [ ]:
asset = 2060

data = DfToData(market_train_df,'returnsClosePrevRaw10')
pca_history = Close10d

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,10),sharey=True)
reconstructed = np.sum(ppca.C[asset]*ppca.stds[asset]*pca_history,axis=1)
ax1.plot(reconstructed)
ax1.plot(data[:,asset])
ax2.plot(data[:,asset]-reconstructed)
print('Data Std Dev       : {:6.4f}'.format(np.nanstd(data[:,asset])))
print('Reconstructed StDev: {:6.4f}'.format(np.nanstd(reconstructed)))
print('Noise StDev        : {:6.4f}'.format(np.nanstd(data[:,asset]-reconstructed)))

So yes, a strong reduction in noise if we look at the returns we trained with.

How about if we use a different set of returns?

In [ ]:
data2 = DfToData(market_train_df,'returnsOpenPrevRaw1')
pca_history = Open1d

ppca.eig_vals
asset = 2060
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,10),sharey=True)
reconstructed2 = np.sum(ppca.C[asset]*ppca.stds[asset]*pca_history,axis=1)
ax1.plot(reconstructed)
ax1.plot(data2[:,asset])
ax2.plot(data2[:,asset]-reconstructed2)
print('Data Std Dev       : {:6.4f}'.format(np.nanstd(data2[:,asset])))
print('Reconstructed StDev: {:6.4f}'.format(np.nanstd(reconstructed2)))
print('Noise StDev        : {:6.4f}'.format(np.nanstd(data2[:,asset]-reconstructed2)))

So not so much on the noise reduction there. Seems like the PCA might just be telling us things about 10 day closing returns.

If we look at the returns on the remaining noise on the training data we see:

In [ ]:
noise = data[:,asset]-reconstructed
plt.hist(noise[~np.isnan(noise)],bins=25)

Still pretty heavy tailed - not that Normal - but still shouldn't be hard to model with a residue term.

However, for this competition getting the size of the noise exactly right isn't really that important. All we really care about is predicting the correct direction. Let's have a look at a scatter plot to see how we've done for that on the training data (remembering this is still just one arbitrary asset):

In [ ]:
plt.scatter(data[:,asset],reconstructed)

Looks promising

And for the one days returns?

In [ ]:
plt.scatter(data2[:,asset],reconstructed2)

Still a clear structure - but not as good as for the 10 day returns.

But this is just the time series for one asset. To confirm the point, lets look at all of them. Firstly for the Series that we used in training.

In [ ]:
def DfWithErrorStats(data,pca_history):
    data_std = []
    recon_std = []
    noise_std = []
    count_nan = []
    correct_count = []
    wrong_count = []
    net_return = []
    for asset in range(data.shape[1]):
        reconstructed = np.sum(ppca.C[asset]*ppca.stds[asset]*pca_history,axis=1)
        data_std.append(np.nanstd(data[:,asset]))
        recon_std.append(np.nanstd(reconstructed))
        noise_std.append(np.nanstd(data[:,asset]-reconstructed))
        count_nan.append((~np.isnan(data[:,asset])).sum())
        correct = 0
        wrong = 0
        net = 0
        for orig,recon in zip(data[:,asset],reconstructed):
            if not np.isnan(orig):
                if ((orig > 0) and (recon > 0)) or ((orig < 0) and (recon < 0)):
                    correct += 1
                else:
                    wrong += 1
                net += np.sign(recon)*orig
        correct_count.append(correct)
        wrong_count.append(wrong)
        net_return.append(net)

    std_df = pd.DataFrame({'Count':count_nan,'DataStd':data_std,'ReconStd':recon_std,'NoiseStd':noise_std, 'Correct':correct_count, 'Wrong':wrong_count, 'Net':net_return})
    std_df['Ratio']=std_df['NoiseStd']/std_df['DataStd']
    std_df['Accuracy']=std_df['Correct']/(std_df['Correct']+std_df['Wrong'])
    del data_std
    del recon_std
    del noise_std
    return std_df

In [ ]:
data = market_train_df.pivot(index='time',columns='assetCode',values='returnsClosePrevRaw10').values
asset_means = np.nanmean(data,axis=0)
data -= asset_means

pca_history = Close10d

stats_train_df = DfWithErrorStats(data,pca_history)
stats_train_df.sort_values(['Count'])

In [ ]:
stats_train_df['Accuracy'].hist(bins=50)

In [ ]:
stats_train_df['Correct'].sum()/(stats_train_df['Correct'].sum()+stats_train_df['Wrong'].sum())

So what we are seeing is basically random accuracy for the assets with just a few observations - no big surprises there - but typically 80%+ accuracy for series that have reasonable sample sizes. Most assets correctly classify if the stock will go up or down in 80% plus of cases. Overall accuracy classified in 84% of cases. 

I did try this for a range of different numbers of components (before applying filtering to the data) - and got the accuracies listed below.  218 seemed to be the maximum components I could fit:
* 218 PC - 88.19%
* 200 PC - 87.9%
* 128 PC - 84.74%
* 100 PC - 82.81%
* 50 PC - 78.78%

Next we look at one of the other input series:

In [ ]:
data = market_train_df.pivot(index='time',columns='assetCode',values='returnsOpenPrevRaw1').values
asset_means = np.nanmean(data,axis=0)
data -= asset_means

pca_history = Open1d

stats_1day_df = DfWithErrorStats(data,pca_history)
stats_1day_df.sort_values(['Count'])

In [ ]:
stats_1day_df['Accuracy'].hist(bins=50)

In [ ]:
stats_1day_df['Correct'].sum()/(stats_train_df['Correct'].sum()+stats_train_df['Wrong'].sum())

So not as good here. Only 75% accuracy overall and most series classified accurately 70% to 80% of the time. Remains to be seen if that will be good enough to use PCAed data for training.

Next lets analyse the volume

In [ ]:
data = market_train_df.pivot(index='time',columns='assetCode',values='volume').values
asset_means = np.nanmean(data,axis=0)
data -= asset_means

pca_history = Volume

stats_vol_df = DfWithErrorStats(data,pca_history)
stats_vol_df.sort_values(['Count'])

In [ ]:
stats_vol_df['Accuracy'].hist(bins=50)

In [ ]:
stats_vol_df['Correct'].sum()/(stats_train_df['Correct'].sum()+stats_train_df['Wrong'].sum())

OK - so that's not so good. The relationship between PCA volume and actual volume (talking about deviations from the mean volume for each asset here) is only slightly better than random - and a sizeable chunk of the assets are worse than random. So PCA volume is out as a driver.

Finally the important one - does this PCA also work on the (forward looking) target values as well as the backwards looking drivers?

In [ ]:
data = market_train_df.pivot(index='time',columns='assetCode',values='returnsOpenNextMktres10').values
asset_means = np.nanmean(data,axis=0)
data -= asset_means

pca_history = Targets

stats_target_df = DfWithErrorStats(data,pca_history)
stats_target_df.sort_values(['Count'])

In [ ]:
stats_target_df['Accuracy'].hist(bins=50)

In [ ]:
stats_target_df['Correct'].sum()/(stats_train_df['Correct'].sum()+stats_train_df['Wrong'].sum())

That's not terrible. Just passes the 2/3 test required for being right as a trader to make money. It is disappointing though - would have expected better when building the principle components off historical 10 day returns.